In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("E:\Downlload\drugsComTest_raw.tsv",error_bad_lines=False, sep='\t')
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [3]:
df.dropna(inplace=True)
df['Sentiment'] = np.where(df['rating'] > 6, 1, 0)
df= df[['review','Sentiment']]
df.head()

,review,Sentiment
0,"""I&#039;ve tried a few antidepressants over th...",1
1,"""My son has Crohn&#039;s disease and has done ...",1
2,"""Quick reduction of symptoms""",1
3,"""Contrave combines drugs that were used for al...",1
4,"""I have been on this birth control for one cyc...",1


In [4]:
df['Sentiment'].value_counts()

1    35250
0    18221
Name: Sentiment, dtype: int64

In [5]:
x = df['review']
y = df['Sentiment']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.1, random_state=0)

In [7]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((48123,), (5348,), (48123,), (5348,))

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import text
from keras.utils import np_utils
from keras.models import Sequential

def label_encoding(y_train):
    """
        Encode the given list of class labels
        :y_train_enc: returns list of encoded classes
        :labels: actual class labels
    """
    lbl_enc = LabelEncoder()
    
    y_train_enc = lbl_enc.fit_transform(y_train)
    labels = lbl_enc.classes_
    
    return y_train_enc, labels


def word_embedding(train, test, max_features, max_len=200):
    try:
        """ Keras Tokenizer class object """
        tokenizer = text.Tokenizer(num_words=max_features)
        tokenizer.fit_on_texts(train)
        
        train_data = tokenizer.texts_to_sequences(train)
        test_data = tokenizer.texts_to_sequences(test)
        
        """ Get the max_len """
        vocab_size = len(tokenizer.word_index) + 1
        
        """ Padd the sequence based on the max-length """
        x_train = sequence.pad_sequences(train_data, maxlen=max_len, padding='post')
        x_test = sequence.pad_sequences(test_data, maxlen=max_len, padding='post')
        """ Return train, test and vocab size """
        return tokenizer, x_train, x_test, vocab_size
    except ValueError as ve:
        raise(ValueError("Error in word embedding {}".format(ve)))

In [9]:
max_features = 6000
max_len = 200

tokenizer, x_pad_train, x_pad_valid, vocab_size = word_embedding(X_train, X_test, max_features)

x_pad_train.shape
print("Vocab size: {}".format(vocab_size))

Vocab size: 32958


In [10]:
x_pad_train.shape,x_pad_valid.shape

((48123, 200), (5348, 200))

In [11]:
from tensorflow import keras
input_length =200
output_length= 200
def build_rnn(vocab_size,output_dim, max_len):

    model = Sequential([
        keras.layers.Embedding(vocab_size,output_dim=output_length,
                              input_length=input_length),
        keras.layers.BatchNormalization(),
        keras.layers.Bidirectional(keras.layers.LSTM(256,return_sequences=True)),
        keras.layers.GlobalMaxPool1D(),
        keras.layers.Dense(256),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(225),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(168),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(148),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(95),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(34),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(32),
        keras.layers.Dense(output_dim, activation='sigmoid')
    ])

    return model

rnn_model = build_rnn(vocab_size, 1, max_len)

rnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 200)          6591600   
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200)          800       
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 512)          935936    
_________________________________________________________________
global_max_pooling1d (Global (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 225)               5

In [12]:
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


history = rnn_model.fit(x_pad_train, 
                        y_train,
                        batch_size=256,
                       epochs=10,
                       verbose=1)

Epoch 1/10
188/188 [==============================] - 2414s 13s/step - loss: 0.5179 - acc: 0.7459
Epoch 2/10
188/188 [==============================] - 2096s 11s/step - loss: 0.3171 - acc: 0.8685
Epoch 3/10
188/188 [==============================] - 1262s 7s/step - loss: 0.2086 - acc: 0.9202
Epoch 4/10
188/188 [==============================] - 1205s 6s/step - loss: 0.1189 - acc: 0.9563
Epoch 5/10
188/188 [==============================] - 1189s 6s/step - loss: 0.0708 - acc: 0.9737
Epoch 6/10
188/188 [==============================] - 1845s 10s/step - loss: 0.0467 - acc: 0.9829
Epoch 7/10
188/188 [==============================] - 2359s 13s/step - loss: 0.0387 - acc: 0.9861
Epoch 8/10
188/188 [==============================] - 1267s 7s/step - loss: 0.0297 - acc: 0.9901
Epoch 9/10
188/188 [==============================] - 1200s 6s/step - loss: 0.0809 - acc: 0.9744
Epoch 10/10
188/188 [==============================] - 1197s 6s/step - loss: 0.0301 - acc: 0.9892


In [13]:
score = rnn_model.evaluate(x_pad_valid, y_test, verbose=1)

print("Loss:%.3f Accuracy: %.3f" % (score[0], score[1]))

168/168 [==============================] - 56s 335ms/step - loss: 0.8549 - acc: 0.8614
Loss:0.855 Accuracy: 0.861


In [14]:
tokenizer.fit_on_sequences(X_test)

X_test = tokenizer.texts_to_sequences(X_test)
x_pad_test = sequence.pad_sequences(X_test, maxlen=max_len, padding='post')

In [15]:
y_preds = rnn_model.predict(x_pad_test)

for arr in y_preds:
    for i in range(len(arr)):
        if arr[i]>0.5:
            arr[i] = 1
        else:
            arr[i] = 0

y_preds = y_preds.astype('int32')

pred_df = pd.DataFrame(y_preds, columns=['pred'])

print(pred_df.shape)
pred_df.head()

(5348, 1)


,pred
0,0
1,0
2,0
3,1
4,1


In [16]:
model_json = rnn_model.to_json()
with open("rnn_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
rnn_model.save_weights("rnn_model.h5")

In [18]:
json_file = open('rnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model =loaded_model.load_weights("rnn_model.h5")
print("Loaded model from disk")

In [19]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [83]:
json_file = open('rnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("rnn_model.h5")
model

In [21]:
import tensorflow
tensorflow.__version__

'2.3.1'

In [22]:
import keras
keras.__version__

'2.4.3'

In [33]:
import flask
flask.__version__

'1.1.2'

In [ ]:
# import pickle

#     # saving
# with open('tokenizer.pickle', 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     # loading
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)